<https://github.com/PolymathicAI/xVal>


In [1]:
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"

In [2]:
import jax.numpy as jnp  # Oddly works in colab to set gpu

arr = jnp.array([1, 2, 3])
arr.devices()

{CpuDevice(id=0)}

In [3]:
import icecream
from icecream import ic

icecream.install()
ic_disable = False
if ic_disable:
    ic.disable()
ic.configureOutput(includeContext=True, contextAbsPath=True)

In [4]:
import os
import ast

from datetime import datetime as dt
from torch.utils.data import Dataset

import hephaestus as hp
import jax
import jax.numpy as jnp
import numpy as np
import optax
import pandas as pd
from flax.training import train_state
from icecream import ic
from jax import random
from flax import struct
from jax.tree_util import tree_flatten
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange

pd.options.mode.copy_on_write = True

In [5]:
def line2df(line, idx):
    data_rows = []
    line = ast.literal_eval(line)
    for i, time_step in enumerate(line["data"]):
        row = {"time_step": i}
        # Add position data for each planet
        for j, position in enumerate(time_step):
            row[f"planet{j}_x"] = position[0]
            row[f"planet{j}_y"] = position[1]
        data_rows.append(row)

    df = pd.DataFrame(data_rows)
    description = line.pop("description")
    step_size = description.pop("stepsize")
    for k, v in description.items():
        for k_prop, v_prop in v.items():
            df[f"{k}_{k_prop}"] = v_prop
    df["time_step"] = df["time_step"] * step_size
    df.insert(0, "idx", idx)

    return df

In [6]:
files = os.listdir("data")
if "planets.parquet" not in files:
    with open("data/planets.data") as f:
        data = f.read().splitlines()

        dfs = []
        for idx, line in enumerate(tqdm(data)):
            dfs.append(line2df(line, idx))
        df = pd.concat(dfs)
    df.to_parquet("data/planets.parquet")
else:
    df = pd.read_parquet("data/planets.parquet")

In [7]:
# Get min, mean, and max number of time steps
df.groupby("idx").count().time_step.agg(["min", "mean", "max"])

min     30.000000
mean    44.511656
max     59.000000
Name: time_step, dtype: float64

In [8]:
class SimpleDS(Dataset):
    def __init__(self, df):
        # Add nan padding to make sure all sequences are the same length
        # use the idx column to group by
        self.max_seq_len = df.groupby("idx").count().time_step.max()

        self.df = df
        self.batch_size = self.max_seq_len

        self.special_tokens = ["[PAD]", "[NUMERIC_MASK]", "[MASK]"]
        self.cat_mask = "[MASK]"
        self.numeric_mask = "[NUMERIC_MASK]"

        self.col_tokens = [col_name for col_name in df.columns if col_name != "idx"]

        self.tokens = self.special_tokens + self.col_tokens

        self.token_dict = {token: i for i, token in enumerate(self.tokens)}
        self.token_decoder_dict = {i: token for i, token in enumerate(self.tokens)}
        self.n_tokens = len(self.tokens)
        self.numeric_indices = jnp.array(
            [self.tokens.index(i) for i in self.col_tokens]
        )

        self.numeric_mask_token = self.tokens.index(self.numeric_mask)

    def __len__(self):
        return df.idx.max() + 1  # probably should be max idx + 1 thanks

    def __getitem__(self, set_idx):
        batch = self.df.loc[
            df.idx == set_idx, [col for col in self.df.columns if col != "idx"]
        ]
        batch = jnp.array(batch.values)
        # Add padding
        batch_len, n_cols = batch.shape
        pad_len = self.max_seq_len - batch_len
        padding = jnp.full((pad_len, n_cols), jnp.nan)
        batch = jnp.concatenate([batch, padding], axis=0)
        return batch

In [9]:
train_ds = SimpleDS(df)

In [10]:
time_series_regressor = hp.simple_time_series.SimplePred(train_ds, d_model=64 * 4)

In [11]:
train_ds[0]

Array([[ 0.        ,  1.5600603 , -0.854437  , ...,         nan,
                nan,         nan],
       [ 0.4651163 ,  1.689858  , -0.5143588 , ...,         nan,
                nan,         nan],
       [ 0.9302326 ,  1.7535888 , -0.15420859, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]], dtype=float32)

In [12]:
def make_batch(ds: SimpleDS, start: int, length: int):
    data = []
    for i in range(start, length + start):
        data.append(ds[i])

    return jnp.array(data)


batch = make_batch(train_ds, 0, 4)

In [13]:
A = jnp.ones((4, 44, 16))  # Example data for A
B = jnp.ones((44, 16, 256))  # Example data for B

# Expand A with a new axis to align with B for broadcasting
A_expanded = A[:, :, :, jnp.newaxis]  # Shape becomes (4, 44, 16, 1)

# Perform element-wise multiplication with broadcasting
result = A_expanded * B  # The resulting shape will be (4, 44, 16, 256)
result.shape

(4, 44, 16, 256)

In [14]:
vars = time_series_regressor.init(random.PRNGKey(0), batch)

x = time_series_regressor.apply(vars, batch)

ic| simple_time_series.py:188 in __call__()
    col_embeddings.shape: (26, 256)
    numeric_inputs.shape: (4, 59, 26)
ic| simple_time_series.py:192 in __call__()
    repeated_numeric_indices.shape: (59, 26)
ic| simple_time_series.py:199 in __call__()
    numeric_col_embeddings.shape: (59, 26, 256)
    numeric_inputs.shape: (4, 59, 26)
ic| simple_time_series.py:200 in __call__()
    numeric_col_embeddings.shape: (59, 26, 256)
    numeric_inputs[:, :, :, None].shape: (4, 59, 26, 1)
ic| simple_time_series.py:215 in __call__()
    kv_embeddings.shape: (4, 59, 26, 256)
ic| simple_time_series.py:217 in __call__()
    kv_embeddings.shape: (4, 59, 26, 256)
    col_embeddings.shape: (26, 256)
ic| simple_time_series.py:220 in __call__()
    out.shape: (4, 59, 26, 256)
ic| simple_time_series.py:224 in __call__()
    out.shape: (4, 59, 26, 272)
ic| simple_time_series.py:240 in __call__()
    out.shape: (4, 59, 26, 272)
ic| simple_time_series.py:251 in __call__()
    f"Second MHA out shape: {out.sh

In [15]:
x.shape

(4, 59, 26)

In [16]:
batch.shape

(4, 59, 26)

In [17]:
def calculate_memory_footprint(params):
    """Calculate total memory footprint of JAX model parameters."""
    total_bytes = 0
    # Flatten the parameter tree structure into a list of arrays
    flat_params, _ = tree_flatten(params)
    for param in flat_params:
        # Calculate bytes: number of elements * size of each element
        bytes_per_param = param.size * param.dtype.itemsize
        total_bytes += bytes_per_param
    return total_bytes

In [18]:
mts_root_key = random.PRNGKey(44)
mts_main_key, ts_params_key, ts_data_key = random.split(mts_root_key, 3)


def clip_gradients(gradients, max_norm):
    total_norm = jnp.sqrt(sum(jnp.sum(jnp.square(grad)) for grad in gradients.values()))
    scale = max_norm / (total_norm + 1e-6)
    clipped_gradients = jax.tree_map(
        lambda grad: jnp.where(total_norm > max_norm, grad * scale, grad), gradients
    )
    return clipped_gradients


# @jax.jit
def calculate_loss(params, state, inputs, dataset: SimpleDS):
    out = state.apply_fn(
        {"params": params},
        hp.mask_tensor(inputs, dataset, prng_key=ts_data_key),
    )
    loss = optax.squared_error(out, inputs).mean()

    return loss


@jax.jit
def eval_step(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss, loss_dict = loss_fn(state.params)
    return loss, loss_dict


@jax.jit
def train_step(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds)

    grad_fn = jax.value_and_grad(loss_fn)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss, grad = grad_fn(state.params)
    # grad = replace_nans(grad)
    # grad = clip_gradients(grad, 1.0)
    state = state.apply_gradients(grads=grad)

    return state, loss


def create_train_state(model, prng, batch, lr):
    params = model.init(prng, batch)
    # optimizer = optax.chain(optax.adam(lr))
    optimizer = optax.chain(optax.clip_by_global_norm(0.4), optax.adam(lr))
    # optimizer_state = optimizer.init(params)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        params=params["params"],
        tx=optimizer,
        # tx_state=optimizer_state,
    )


batch_size = 2
# batch = train_ds[0]

state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)

ic| simple_time_series.py:188 in __call__()
    col_embeddings.shape: (26, 256)
    numeric_inputs.shape: (4, 59, 26)
ic| simple_time_series.py:192 in __call__()
    repeated_numeric_indices.shape: (59, 26)
ic| simple_time_series.py:199 in __call__()
    numeric_col_embeddings.shape: (59, 26, 256)
    numeric_inputs.shape: (4, 59, 26)
ic| simple_time_series.py:200 in __call__()
    numeric_col_embeddings.shape: (59, 26, 256)
    numeric_inputs[:, :, :, None].shape: (4, 59, 26, 1)
ic| simple_time_series.py:215 in __call__()
    kv_embeddings.shape: (4, 59, 26, 256)
ic| simple_time_series.py:217 in __call__()
    kv_embeddings.shape: (4, 59, 26, 256)
    col_embeddings.shape: (26, 256)
ic| simple_time_series.py:220 in __call__()
    out.shape: (4, 59, 26, 256)
ic| simple_time_series.py:224 in __call__()
    out.shape: (4, 59, 26, 272)
ic| simple_time_series.py:240 in __call__()
    out.shape: (4, 59, 26, 272)
ic| simple_time_series.py:251 in __call__()
    f"Second MHA out shape: {out.sh

In [19]:
writer_name = "SimpleTS"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")

train_summary_writer = SummaryWriter(
    "runs/" + writer_time + "_" + writer_name + "_train"
)
test_summary_writer = SummaryWriter("runs/" + writer_time + "_" + writer_name + "_test")


def create_iterations(ds, batch_size, prng_key):
    arr = jnp.arange(len(ds))

    # Shuffle the array
    shuffled_arr = jax.random.permutation(prng_key, arr)
    num_groups = (
        arr.size + (batch_size - 1)
    ) // batch_size  # This ensures that the last group has <= batch_size elements

    # Split the shuffled array into groups
    groups = jnp.split(
        shuffled_arr, jnp.arange(batch_size, batch_size * num_groups, batch_size)
    )
    return groups


test_set_key = random.PRNGKey(4454)


batch_count = 0
for j in trange(14, desc="epochs"):
    arrs = create_iterations(train_ds, 2, ts_data_key)
    for i in tqdm(arrs, leave=False, desc="batches"):
        # for i in trange(len(pre_train) // batch_size, leave=False):
        # for i in trange(len(pre_train) // batch_size //10, leave=False):
        batch = make_batch(train_ds, i[0], 4)

        state, loss = train_step(state, batch)
        batch_count += 1

        if batch_count % 10 == 0:
            train_summary_writer.add_scalar(
                "loss/loss", np.array(loss.item()), batch_count
            )

epochs:   0%|          | 0/14 [00:00<?, ?it/s]

batches:   0%|          | 0/62500 [00:00<?, ?it/s]

ic| simple_time_series.py:188 in __call__()
    col_embeddings.shape: (26, 256)
    numeric_inputs.shape: (4, 59, 26)
ic| simple_time_series.py:192 in __call__()
    repeated_numeric_indices.shape: (59, 26)
ic| simple_time_series.py:199 in __call__()
    numeric_col_embeddings.shape: (59, 26, 256)
    numeric_inputs.shape: (4, 59, 26)
ic| simple_time_series.py:200 in __call__()
    numeric_col_embeddings.shape: (59, 26, 256)
    numeric_inputs[:, :, :, None].shape: (4, 59, 26, 1)
ic| simple_time_series.py:215 in __call__()
    kv_embeddings.shape: (4, 59, 26, 256)
ic| simple_time_series.py:217 in __call__()
    kv_embeddings.shape: (4, 59, 26, 256)
    col_embeddings.shape: (26, 256)
ic| simple_time_series.py:220 in __call__()
    out.shape: (4, 59, 26, 256)
ic| simple_time_series.py:224 in __call__()
    out.shape: (4, 59, 26, 272)
ic| simple_time_series.py:240 in __call__()
    out.shape: (4, 59, 26, 272)
ic| simple_time_series.py:251 in __call__()
    f"Second MHA out shape: {out.sh

In [ ]:
arrs

NameError: name 'arrs' is not defined

In [ ]:
ts_data_key = random.PRNGKey(1)
ts_data_key, new_key = random.split(ts_data_key)

In [ ]:
# for i in range(4):
#     new_key, subkey = random.split(new_key)
#     print(subkey, new_key)
#     random_index = random.randint(new_key, (1,), 0, len(test_ds) - 1).item()
#     print(random_index)

In [ ]:
model_name = f"big_train_{dt.now()}_"

current_dir = os.getcwd()

if not os.path.exists("./pre_trained_models/"):
    os.makedirs("./pre_trained_models/")

path = os.path.join(current_dir, "./pre_trained_models/")


ckpt_dir = f"./pre_trained_models/{model_name}"

# checkpoints.save_checkpoint(
#     ckpt_dir=path, target=state, step=batch_count, overwrite=True, prefix=model_name
# )

In [ ]:
def jax_array_memory_usage(array):
    """Calculate the memory usage of a JAX array in bytes."""
    # Get the number of bytes per element based on the data type
    bytes_per_element = array.dtype.itemsize
    # Calculate the total number of elements in the array
    total_elements = np.prod(array.shape)
    # Calculate total memory usage
    memory_usage_bytes = bytes_per_element * total_elements
    return memory_usage_bytes


cat_memory_usage = jax_array_memory_usage(batch.categorical)
num_memory_usage = jax_array_memory_usage(batch.numeric)
memory_usage = cat_memory_usage + num_memory_usage
memory_usage_gb = memory_usage / 1024 / 1024 / 1024
print(f"Memory usage: {memory_usage} bytes")
print(f"Memory usage: {memory_usage_gb} gb")